# INSTALL HUGGINGSOUND LIBRARY AND IMPORT SPEECH RECOGNITION MODEL

In [ ]:
!pip install huggingsound
from huggingsound import SpeechRecognitionModel

# SELECT THE SPEECH RECOGNITION MODEL 

In [3]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
#jonatasgrosman/wav2vec2-large-xlsr-53-spanish
#tyoc213/wav2vec2-large-xlsr-nahuatl

05/04/2022 01:25:08 - INFO - huggingsound.speech_recognition.model - Loading model...


# COLLECT AUDIO FROM MIC

In [4]:
!pip install ffmpeg-python
from scipy.io.wavfile import write

In [5]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

# INSTALL LIBRARIES FOR VOICE COMMANDS IN QUANTUM TIC TAC TOE

In [6]:
!sudo apt-get install libportaudio2
!pip install sounddevice
!pip install wavio
!pip install scipy
!pip install NumPy
!pip install spa2num
from spa2num.converter import to_number

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


# QUANTUM TIC TAC TOE GAME

In [ ]:
!pip install qiskit
RESTART_HTML = """
<script>

//document.body.appendChild(my_div);
document.body.removeChild(document.body.lastElementChild); 

</script>
"""

In [ ]:
from termcolor import colored, cprint
import json
from qiskit import *
from qiskit.tools.monitor import job_monitor
# Define the board

def resetBoard():
  return {'1': [' ', 0] , '2': [' ', 0], '3': [' ', 0],
          '4': [' ', 0], '5': [' ', 0], '6': [' ', 0],
          '7': [' ', 0], '8': [' ', 0], '9': [' ', 0]}

def printBoard(board):
  print()
  colour = 0
  for i in range (1,10):
    if board[str(i)][1] == 0:
      cprint(board[str(i)][0], end='')
    else:
      if (colour == 0 or colour == 1):
        cprint(board[str(i)][0], 'red', end='')
        colour = colour + 1
      elif (colour == 2 or colour == 3):
        cprint(board[str(i)][0], 'green', end='')
        colour = colour + 1
      elif (colour == 4 or colour == 5):
        cprint(board[str(i)][0], 'blue', end='')
        colour = colour + 1
      elif (colour == 6 or colour == 7):
        cprint(board[str(i)][0], 'yellow', end='')
        colour = colour + 1

    if i % 3 == 0:
      print()
      if i != 9: 
        print('-+-+-')
    else:
      cprint('|', end='') 

def make_classic_move(theBoard, turn, count, circuit):

  valid_move = 0
  valid_moves = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
  
  while (not valid_move):
    print()
    print("¿En dónde? (1-9) ", end='')
    freq = 16000
    audio, sr = get_audio()
    write("recordingchoice.wav", 44100, audio)

        # Convert the NumPy array to audio file
    audio_paths = ["./recordingchoice.wav"]
    model.transcribe(audio_paths)
    c=model.transcribe(audio_paths)[0]
    first_value = list(c.values())[0]
        

    location = str(to_number(first_value))

#    location = input()

    if location in valid_moves:
  
        
      if theBoard[location][0] == ' ':
        valid_move = 1
        # set the location's marker
        theBoard[location][0] = turn
        # increment counter (total markers on board) *when this = 9, collapse the board, also called measurement
        count += 1
        # set marker's state (classical or quantum)
        theBoard[location][1] = 0 # classical (not flashing on screen)

        # set qubit[location] to ON, 100% = 1
        # one pauli X gate
        circuit.x(int(location)-1)

        print(circuit.draw())
      else: 
        print()
        print("Es lugar ya está ocupado.")
    else:
      print("Por favor selecciona una celda del 1-9")

  return theBoard, turn, count, circuit

def make_quantum_move(theBoard, count, circuit, turn):

  valid_move = False
  valid_moves = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]

  while (not valid_move):
      
    print()
    print("¿En dónde? (1-9) ")
    freq = 16000
    audio, sr = get_audio()
    write("recordingchoice.wav", 44100, audio)

        # Convert the NumPy array to audio file
    audio_paths = ["./recordingchoice.wav"]
    model.transcribe(audio_paths)
    c=model.transcribe(audio_paths)[0]
    first_value = list(c.values())[0]
        

    location1 = str(to_number(first_value))
    display(HTML(RESTART_HTML)) 
#    location1 = input()
    print("¿En dónde? (1-9) ")
    freq = 16000
    audio, sr = get_audio()
    write("recordingchoice.wav", 44100, audio)

        # Convert the NumPy array to audio file
    audio_paths = ["./recordingchoice.wav"]
    model.transcribe(audio_paths)
    c=model.transcribe(audio_paths)[0]
    first_value = list(c.values())[0]
        

    location2 = str(to_number(first_value))
    display(HTML(RESTART_HTML)) 
#    location2 = input()

    if theBoard[location1][0] == ' ' and theBoard[location2][0] == ' ' and location1 != location2:
        # set the location's marker
        theBoard[location1][0] = turn
        theBoard[location2][0] = turn
        # increment counter (total markers on board) *when this = 9, collapse the board, also called measurement
        count += 2
        # set marker's state (classical or quantum)
        theBoard[location1][1] = 1 # quantum (flashing on screen)
        theBoard[location2][1] = 1 # quantum (flashing on screen)


        # set qubit[location1], qubit[location2] to superposition/entangled

        # hadamard gates
        circuit.h(int(location1)-1)

        # x gate
        circuit.x(int(location2)-1)

        # cnot gate
        circuit.cx(int(location1)-1,int(location2)-1)

        print(circuit.draw())
        valid_move = True
    else:
        print()
        print("Has seleccionado una posición no permitida")
    

  return theBoard, count, circuit, turn

def measure(circuit, theBoard, count):
  # trigger collapse
  printBoard(theBoard)
  print()
  print("Colapso desencadenado.")
  print()

  # Use Aer's qasm_simulator
  simulator = Aer.get_backend('qasm_simulator')

  circuit.measure(0,0)
  circuit.measure(1,1)
  circuit.measure(2,2)
  circuit.measure(3,3)
  circuit.measure(4,4)
  circuit.measure(5,5)
  circuit.measure(6,6)
  circuit.measure(7,7)
  circuit.measure(8,8)

  print(circuit.draw())

  # Execute the circuit on quantum simulator
  job = execute(circuit, simulator, shots=1)

  # Grab results from the job
  result = job.result()

  out = json.dumps(result.get_counts()) #Converts the result.get_counts() into a string
  string = out[2:11] #Removes unnecessary data from string, leaving us with board

  # update board
  for i in range(9):
      if string[i] == '1':
          # cement value in the board
          theBoard[str(9-i)][1] = 0
      else:
          # make square empty
          theBoard[str(9-i)][1] = 0
          theBoard[str(9-i)][0] = ' '

  # update count (total number of markers on the board)
  count = 0
  for i in range(9):
      theBoard[str(i+1)][1] = 0
      if theBoard[str(i+1)][0] != ' ':
          count += 1

  # reset qubits
  circuit.reset(0)
  circuit.reset(1)
  circuit.reset(2)
  circuit.reset(3)
  circuit.reset(4)
  circuit.reset(5)
  circuit.reset(6)
  circuit.reset(7)
  circuit.reset(8)

  for i in range(9):
      if string[8-i] == '1':
          # add pauli x gate
          circuit.x(i)

  return circuit, string, theBoard, count

def check_win(theBoard, turn):
  if theBoard['7'][0] == theBoard['8'][0] == theBoard['9'][0] != ' ': # across the top
      if theBoard['7'][1] == theBoard['8'][1] == theBoard['9'][1] == 0: # only cemented markers
          printBoard(theBoard)
          print("\nJuego terminado.\n")                
          print(" **** ", end='')
          print(theBoard['8'][0], end='')
          print(" ganó ****")
          print() 
          return True

  elif theBoard['4'][0] == theBoard['5'][0] == theBoard['6'][0] != ' ': # across the middle
      if theBoard['4'][1] == theBoard['5'][1] == theBoard['6'][1] == 0: # only cemented markers
          printBoard(theBoard)
          print("\nJuego terminado.\n")                
          print(" **** ", end='')
          print(theBoard['5'][0], end='')
          print(" ganó ****")
          print()
          return True

  elif theBoard['1'][0] == theBoard['2'][0] == theBoard['3'][0] != ' ': # across the bottom
      if theBoard['1'][1] == theBoard['2'][1] == theBoard['3'][1] == 0: # only cemented markers
          printBoard(theBoard)
          print("\nJuego terminado.\n")                
          print(" **** ", end='')
          print(theBoard['2'][0], end='')
          print(" ganó ****")
          print()
          return True

  elif theBoard['1'][0] == theBoard['4'][0] == theBoard['7'][0] != ' ': # down the left side
      if theBoard['1'][1] == theBoard['4'][1] == theBoard['7'][1] == 0: # only cemented markers
          printBoard(theBoard)
          print("\nJuego terminado.\n")                
          print(" **** ", end='')
          print(theBoard['4'][0], end='')
          print(" ganó ****")
          print()
          return True

  elif theBoard['2'][0] == theBoard['5'][0] == theBoard['8'][0] != ' ': # down the middle
      if theBoard['2'][1] == theBoard['5'][1] == theBoard['8'][1] == 0: # only cemented markers
          printBoard(theBoard)
          print("\nJuego terminado.\n")                
          print(" **** ", end='')
          print(theBoard['5'][0], end='')
          print(" ganó ****")
          print()
          return True

  elif theBoard['3'][0] == theBoard['6'][0] == theBoard['9'][0] != ' ': # down the right side
      if theBoard['3'][1] == theBoard['6'][1] == theBoard['9'][1] == 0: # only cemented markers
          printBoard(theBoard)
          print("\nJuego terminado.\n")                
          print(" **** ", end='')
          print(theBoard['6'][0], end='')
          print(" ganó ****")
          print()
          return True

  elif theBoard['7'][0] == theBoard['5'][0] == theBoard['3'][0] != ' ': # diagonal
      if theBoard['7'][1] == theBoard['5'][1] == theBoard['3'][1] == 0: # only cemented markers
          printBoard(theBoard)
          print("\nJuego terminado.\n")                
          print(" **** ", end='')
          print(theBoard['5'][0], end='')
          print(" ganó ****")
          print()
          return True

  elif theBoard['1'][0] == theBoard['5'][0] == theBoard['9'][0] != ' ': # diagonal
      if theBoard['1'][1] == theBoard['5'][1] == theBoard['9'][1] == 0: # only cemented markers
          printBoard(theBoard)
          print("\nJuego terminado.\n")                
          print(" **** ", end='')
          print(theBoard['5'][0], end='')
          print(" ganó ****")
          print()
          return True

#Implementation of Two Player Tic-Tac-Toe game in Python.
# start game function
# Now we'll write the main function which has all the gameplay functionality.


def game():

    turn = 'X'
    count = 0
    win = False
    x_collapse = 1
    y_collapse = 1

    # initialise quantum circuit with 9 qubits (all on OFF = 0)
    circuit = QuantumCircuit(9, 9)

    while (not win):

        # ============================= ROUND START ============================ 
        global theBoard
        printBoard(theBoard)

        print()
        print("Es tu turno " + turn + ". ¿Quieres hacer un (1) movimiento clásico, (2) movimiento cuántico, (3) colapso?, o (4) quitar?")
        freq = 16000
        audio, sr = get_audio()
        write("recordingchoice.wav", 44100, audio)

        # Convert the NumPy array to audio file
        audio_paths = ["./recordingchoice.wav"]
        model.transcribe(audio_paths)
        c=model.transcribe(audio_paths)[0]
        first_value = list(c.values())[0]
        

        move = str(to_number(first_value))
        display(HTML(RESTART_HTML))  

#        move = input()

        # ============================= CLASSIC MOVE ===========================

        if int(move) == 1:
            theBoard, turn, count, circuit = make_classic_move(theBoard, turn, count, circuit)
            madeMove = True

        # ============================= QUANTUM MOVE ===========================

        elif int(move) == 2 and count > 8:
          # cant do a quantum move if there's only 1 empty square left
          print()
          print("¡No hay espacios suficientes para eso!")

        elif int(move) == 2 and count < 8:
          theBoard, count, circuit, turn = make_quantum_move(theBoard, count, circuit, turn)
          madeMove = True
        
        # ============================= COLLAPSE/MEASURE =======================

        elif int(move) == 3:

          if (turn == 'X' and x_collapse== 1 ):
            circuit, string, theBoard, count = measure(circuit, theBoard, count)
            x_collapse = 0
          elif (turn == 'O' and y_collapse == 1):
            circuit, string, theBoard, count = measure(circuit, theBoard, count)
            y_collapse = 0
          else:
            print("¡Ya has utilizado tu colapso en este movimiento!")

        # ============================= QUIT ===================================

        elif int(move) == 4:
            break
        
        # ============================= CHECK FOR WIN ==========================

        # Now we will check if player X or O has won,for every move  
        if count >= 5:
          win = check_win(theBoard, turn)
          if (win):
            break



        # If neither X nor O wins and the board is full, we'll declare the result as 'tie'.
        if count == 9:
          circuit, string, theBoard, count = measure(circuit, theBoard, count)
          win = check_win(theBoard, turn)
          if count == 9:
            print("\nJuego terminado.\n")                
            print("¡Es un empate !")
            print()
            win = True
          


        # Now we have to change the player after every move.
        if  (madeMove):  
          madeMove = False
          if turn =='X':
              turn = 'O'
          else:
              turn = 'X'  
          display(HTML(RESTART_HTML))       
    

    # Now we will ask if player wants to restart the game or not.
    restart = input("¿Quieres jugar de nuevo?(Si/No) ")
    if restart == "Si" or restart == "si":

        theBoard = resetBoard()
        game()

def start_menu():
    start_menu = """
    Menú inicial:
    1. Empezar a jugar
    2. Cómo jugar
    3. Quitar
    """ 
    
    print("""
    ###########################
    ### Gato cuántico ###
    ###########################
    """)
    print(start_menu)
    choice = 0
    while (choice != '1'):
      print("¿Que te gustaría hacer? ", end='')
      freq = 16000
  
        # # Recording duration
        # duration = 5
  
        # # Start recorder with the given values 
        # # of duration and sample frequency
        # recording = sd.rec(int(duration * freq), 
        #            samplerate=freq, channels=2)
  
        # # Record audio for the given number of seconds
        # sd.wait()

        # # This will convert the NumPy array to an audio
        #     # file with the given sampling frequency
        # write("recordingprueba.wav", freq, recording) 

        # aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

      audio, sr = get_audio()
      write("recordingprueba.wav", 44100, audio)

        # aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

        # Convert the NumPy array to audio file
      audio_paths = ["./recordingprueba.wav"]
      model.transcribe(audio_paths)
      b=model.transcribe(audio_paths)[0]
      first_value = list(b.values())[0]
        

      choice = str(to_number(first_value)) 


#      choice = input()

      if (choice == '2'):
        How_To = """ 
        En el juego del gato cuántico, cada celda empieza vacía y tu meta es crear una línea de tres con tus O's/X's.
        Si juegas un movimiento clásico la celda que escojas será permanente.
        Si juegas un movimiento cuántico se creará un estado superpuesto entre dos celdas que escojas. Sólo puedes completar un movimiento cuántico con dos celdas vacías.
        El tablero "colapsará" una vez que se llene. En el colapso, cada superposición de estados escogerá solo una posición por probabilidad.
        Cada jugador puede decidir colapsar el tablero prematuramente, y esto se puede hacer una vez por ronda.
        """
        print(How_To)

      if (choice == '3'):
        print("¡Nos vemos!")
        break
      
    return choice
    display(HTML(RESTART_HTML)) 

#Reset the board at start
theBoard = resetBoard()

#Set no moves made yet
if (start_menu() == '1'):  
  madeMove = False
  game()


    ###########################
    ### Gato cuántico ###
    ###########################
    

    Menú inicial:
    1. Empezar a jugar
    2. Cómo jugar
    3. Quitar
    
¿Que te gustaría hacer? 

100%|██████████| 1/1 [00:02<00:00,  2.25s/it]


 | | 
-+-+-
 | | 
-+-+-
 | | 

Es tu turno X. ¿Quieres hacer un (1) movimiento clásico, (2) movimiento cuántico, (3) colapso?, o (4) quitar?


100%|██████████| 1/1 [00:03<00:00,  3.34s/it]



¿En dónde? (1-9) 


...

¿En dónde? (1-9) 


100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


     ┌───┐     
q_0: ┤ H ├──■──
     └───┘  │  
q_1: ───────┼──
     ┌───┐┌─┴─┐
q_2: ┤ X ├┤ X ├
     └───┘└───┘
q_3: ──────────
               
q_4: ──────────
               
q_5: ──────────
               
q_6: ──────────
               
q_7: ──────────
               
q_8: ──────────
               
c: 9/══════════
               



X| |X
-+-+-
 | | 
-+-+-
 | | 

Es tu turno O. ¿Quieres hacer un (1) movimiento clásico, (2) movimiento cuántico, (3) colapso?, o (4) quitar?


100%|██████████| 1/1 [00:03<00:00,  3.00s/it]
